In [1]:
import sys
%cd ..
sys.path.append('.')

/home/oleg/yodas_conversion


In [2]:
from pathlib import Path
import pickle
from typing import Literal, Any
from dataclasses import dataclass

import numpy as np
import pandas as pd
from datasets import load_from_disk, Dataset, concatenate_datasets
from tqdm.auto import tqdm
from scipy.special import log_softmax, expit
import IPython.display
import matplotlib.pyplot as plt

from src.spectrogram_transformer import AudioSpectrogramTransformer, SEGMENT_SHIFT, SEGMENT_LENGTH
from src.yodas_search import YodasSearch, YodasSearchResult
from src.audioset_utils import AudioSetOnthology, sliding_window_mean, Features
from src.speaker_diarization import Segment, get_speech_mask, reorder_speakers

In [3]:
yodas2 = concatenate_datasets([
    (
        concatenate_datasets([load_from_disk(path) for path in tqdm(sorted(Path(dataset_path).glob('*-of-*')))])
        .map(lambda s: {'audio': {'path': dataset_path + '/' + s['audio']['path']}})
    )
    for dataset_path in [
        'datasets/yodas2_ru000_32k',
        'datasets/yodas2_ru001_32k'
    ]
])

# yodas2 = load_from_disk('datasets/yodas2_ru000_32k/00000-of-00500')

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/287 [00:00<?, ?it/s]

In [4]:
searcher = YodasSearch(
    dataset=yodas2,
    onthology=AudioSetOnthology(),
    cache_file='tmp/search_features.pkl',
    rewrite_cache=True
)

Extracting search features:   0%|          | 0/52553 [00:00<?, ? examples/s]

Reducing features:   0%|          | 0/52553 [00:00<?, ?it/s]

Reducing features:   0%|          | 0/52553 [00:00<?, ?it/s]

Reducing features:   0%|          | 0/52553 [00:00<?, ?it/s]

Reducing features:   0%|          | 0/52553 [00:00<?, ?it/s]

In [8]:
searcher._features

{'text': array(['Привет друзья и с вами снова Жека у себя на балконе и Давайте опять послушаем короткие анекдот от меня да да друзья вам не послышалось именно от меня от вашего Джеки Поэтому заранее ставим такой вот жирный лайкос Можно два и подписываемся на канал а канал ещё раз называется Жека друзья идёт с работы парень у него в сумке баллон золотой краски Ну и навстречу ему цыганка так надо подходит и говорит ну парень не долго думая достаёт из сумки баллончики поливать и по рукам цыганка мне-то умение с криками набрасывают на этого парня и говорит в ответ ну вы же просили меня позолотить вам ручку',
        'Салам алекуйм Дорогой Шамиль Дорогой Шамиль, мы хотим выразить огромную благодарность Хотим выразить огромную благодарность. Он болел 7 лет Хочет сказать огромное спасибо что он вылечился что вы вылечили его, и что он будет всем рекомендовать Вас. И даже если где-то в метро увидит человека, он обязательно об этом расскажет. Спасибо огромное! Спасибо ! ! !',
        'всем приве